### KNN算法流程

一般情况下，KNN有如下流程:
* 收集数据:确定训练样本集合测试数据；
* 计算测试数据和训练样本集中每个样本数据的距离:
    * 常用的距离计算公式:
        * 欧式距离:$d(x, y)=\sqrt{\sum_{i=1}^{n}\left(x_{i}-y_{i}\right)^{2}}$
        * 曼哈顿距离:$d(x, y)=\sum_{i=1}^{n}\left|x_{i}-y_{i}\right|$
* 按照距离递增的顺序排序;
* 选取距离最近的k个点；
* 确定这k个点中分类信息的频率;
* 返回前k个点中出现频率最高的分类，作为当前测试数据的分类。

### KNN算法从零实现

这里还是以前一天的约会网站为例。

In [1]:
import numpy as np
import pandas as pd

In [2]:
#定义KNN算法模型
class KNeighborsClassifier():
    #这里选择欧式距离作为距离的定义
    def __init__(self,k=3):
        self.k=k
        self.X=None
        self.Y=None
    
    def fit(self,X,Y):
        self.X=X
        self.Y=Y
    
    def predict(self,X_test):
        n,m=X_test.shape
        dataSetSize=self.X.shape[0]
        y_pred=[]
        for i in range(n):  #对每个样本单独预测
            diff=np.tile(X_test[i],(dataSetSize,1))-self.X
            sqDiff=diff**2
            sqDistances=np.sum(sqDiff,axis=1)
            distances=np.squeeze(sqDistances**0.5)  #计算欧式距离
            distancesIndice=distances.argsort()
            #选择距离最近的K个值
            classCount={}
            for i in range(self.k):
                label=self.Y[distancesIndice[i]]
                classCount[label]=classCount.get(label,0)+1
            #排序
            sortedClassCount=sorted(classCount.items(),key=lambda item:item[1],reverse=True)
            y_pred.append(sortedClassCount[0][0])
        
        return np.array(y_pred)

In [3]:
#读取数据
dataset=pd.read_table("datasets/datingTestSet.txt",sep="\t")
X=dataset.iloc[:,:3].values
Y=dataset.iloc[:,3].values

In [4]:
#将文本数据转为数值数据
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
encoder=LabelEncoder()
Y=encoder.fit_transform(Y)

#将训练集数据归一化
scaler=MinMaxScaler()
X=scaler.fit_transform(X)

In [5]:
#划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [6]:
#训练模型
model=KNeighborsClassifier(k=10)
model.fit(X_train,Y_train)

In [7]:
#预测
y_pred=model.predict(X_test)

In [8]:
#结果可视化
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,y_pred)
print(cm)

[[67  2  1]
 [ 1 56  3]
 [ 1  1 68]]
